In [1]:
import pandas as pd
import math
import time
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import json
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

In [2]:
data_eval = pd.read_csv("data/evaluation.csv")
data = pd.read_csv("data/train.csv")
nlp = spacy.load('en_core_web_sm')

c:\users\julie\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


## Verifier user

Boolean to 0 or 1

In [ ]:
#replace the boolean by its int value
df=pd.DataFrame(data_eval["user_verified"])
data_eval["user_verified"]=df.applymap(lambda x: int(x))

df=pd.DataFrame(data["user_verified"])
data["user_verified"]=df.applymap(lambda x: int(x))

## Tweet with # and number of #

In [ ]:
def get_hashtag(x) :
    x = str(x) 
    if "#" in x :
        return 1
    return 0

def get_number_hashtag(x) :
    x = str(x)
    count = 0
    for k in x :
        if k == "#" :
            count+=1
    return count

df=pd.DataFrame(data["text"])
data["num_hashtag"] = df.applymap(lambda x: get_number_hashtag(x))
data["got_hashtag"] = df.applymap(lambda x: get_hashtag(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_hashtag"] = df.applymap(lambda x: get_number_hashtag(x))
data_eval["got_hashtag"] = df.applymap(lambda x: get_hashtag(x))

## Tweet with @ and number of

In [ ]:
def get_at(x) :
    x = str(x) 
    if "@" in x :
        return 1
    return 0

def get_number_at(x) :
    x = str(x)
    count = 0
    for k in x :
        if k == "@" :
            count+=1
    return count

df=pd.DataFrame(data["text"])
data["num_at"] = df.applymap(lambda x: get_number_at(x))
data["got_at"] = df.applymap(lambda x: get_at(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_at"] = df.applymap(lambda x: get_number_at(x))
data_eval["got_at"] = df.applymap(lambda x: get_at(x))

## Tweet with links and number of

In [ ]:
def get_link(x) :
    x = str(x) 
    if "https" in x :
        return 1
    return 0

def get_number_link(x) :
    x = str(x)
    count = x.count("https")
    return count

df=pd.DataFrame(data["text"])
data["num_link"] = df.applymap(lambda x: get_number_link(x))
data["got_link"] = df.applymap(lambda x: get_link(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_link"] = df.applymap(lambda x: get_number_link(x))
data_eval["got_link"] = df.applymap(lambda x: get_link(x))

## Tweet length

In [ ]:
def get_lenght(x) :
    return len(str(x))

df=pd.DataFrame(data["text"])
data["length"] = df.applymap(lambda x: get_lenght(x))

df=pd.DataFrame(data_eval["text"])
data_eval["length"] = df.applymap(lambda x: get_lenght(x))

## Contains RT or similar

In [ ]:
def contains_RT(x) :
    liste = ["rt","retweet"]
    x = str(x).lower()
    for k in liste :
        if k in x :
            return 1
    return 0

df=pd.DataFrame(data["text"])
data["contains_rt"] = df.applymap(lambda x: contains_RT(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_rt"] = df.applymap(lambda x: contains_RT(x))

## Followers, friends and status count

Average and center these data.

In [ ]:
cols=["user_statuses_count","user_followers_count","user_friends_count","length"]
for c in cols:
    df=pd.DataFrame(data[c])
    m=data[c].mean()
    v=math.sqrt(data[c].var())
    data[c]=df.applymap(lambda x: (x-m)/v)
    
    # the same mean and sigma
    df=pd.DataFrame(data_eval[c])
    data_eval[c]=df.applymap(lambda x: (x-m)/v)

## Posted at noon or eve

In [5]:
def weak(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [7,8,9,10,11,12] :
        return 1
    return 0

def strong(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [0,1] :
        return 1
    return 0

def zeros_pic(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [17,18,19,20,21,22,23] :
        return 1
    return 0

df=pd.DataFrame(data["timestamp"])
data["weak"] = df.applymap(lambda x: weak(x))
data["strong"] = df.applymap(lambda x: strong(x))
data["zeros_pic"] = df.applymap(lambda x: zeros_pic(x))

df=pd.DataFrame(data_eval["timestamp"])
data_eval["weak"] = df.applymap(lambda x: weak(x))
data_eval["strong"] = df.applymap(lambda x: strong(x))
data_eval["zeros_pic"] = df.applymap(lambda x: zeros_pic(x))

## Some words are upper

In [ ]:
def is_upper(x) :
    x = str(x)
    x = x.split(" ")
    for k in x :
        if k.isupper() :
            return 1
    return 0

df=pd.DataFrame(data["text"])
data["is_upper"] = df.applymap(lambda x: is_upper(x))

df=pd.DataFrame(data_eval["text"])
data_eval["is_upper"] = df.applymap(lambda x: is_upper(x))

## Contains ! 

In [ ]:
def contains_excl(x) :
    if "!" in str(x) :
        return 1
    return 0

df=pd.DataFrame(data["text"])
data["contains_excl"] = df.applymap(lambda x: contains_excl(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_excl"] = df.applymap(lambda x: contains_excl(x))

## Contains organisation or location or person

In [ ]:
passage = [-1] 
def contains_per_org_loc(x):
    x = nlp(x)
    per = 0
    org = 0
    gpe = 0
    passage[0] +=1
    if passage[0]%10000 == 0 :
        print(passage[0])
    for k in x.ents :
        if k.label_ == "PERSON" :
            per = 1
        if k.label_ == "ORG" :
            org = 1
        if k.label_ == "GPE" :
            gpe = 1
    return str(per) + " " + str(org) + " " + str(gpe)


df=pd.DataFrame(data["text"])
data["contains_per_org_loc"] = df.applymap(lambda x: contains_per_org_loc(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_per_org_loc"] = df.applymap(lambda x: contains_per_org_loc(x))

In [ ]:
def contains_per(x):
    return str(x).split(" ")[0]

def contains_org(x):
    return str(x).split(" ")[1]

def contains_gpe(x):
    return str(x).split(" ")[2]

df=pd.DataFrame(data["contains_per_org_loc"])
data["contains_per"] = df.applymap(lambda x: contains_per(x))
data["contains_org"] = df.applymap(lambda x: contains_org(x))
data["contains_gpe"] = df.applymap(lambda x: contains_gpe(x))

df=pd.DataFrame(data_eval["contains_per_org_loc"])
data_eval["contains_per"] = df.applymap(lambda x: contains_per(x))
data_eval["contains_org"] = df.applymap(lambda x: contains_org(x))
data_eval["contains_gpe"] = df.applymap(lambda x: contains_gpe(x))

## Date : month, day, hour

In [ ]:
#extract week day and hour from the timestamp
# one hot encoding
def preprocess_month(timestamp) :
    dico_month = {
        "Jan" : 0,
        "Feb" : 1,
        "Mar" : 2,
        "Apr" : 3,
        "May" : 4,
        "Jun" : 5,
        "Jul" : 6,
        "Aug" : 7,
        "Sep" : 8,
        "Oct" : 9,
        "Nov" : 10,
        "Dec" : 11
    }
    res = 12*[0]
    res[dico_month[time.ctime(int(str(timestamp)[:-3]))[4:7]]]=1
    return res

def preprocess_day(timestamp):
    dico_day={"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}
    res=7*[0]
    res[dico_day[time.ctime(int(str(timestamp)[:-3]))[:3]]]=1
    return res

def preprocess_time(timestamp):
    res=24*[0]
    res[int(time.ctime(int(str(timestamp)[:-3]))[11:13])]=1
    return res

df=pd.DataFrame(data["timestamp"])
data["day"] = df.applymap(lambda x: preprocess_day(x))
data["hour"] = df.applymap(lambda x: preprocess_time(x))
data["month"] = df.applymap(lambda x: preprocess_month(x))

df=pd.DataFrame(data_eval["timestamp"])
data_eval["day"] = df.applymap(lambda x: preprocess_day(x))
data_eval["hour"] = df.applymap(lambda x: preprocess_time(x))
data_eval["month"] = df.applymap(lambda x: preprocess_month(x))

## Drop the rest

In [ ]:
to_drop = ["timestamp","user_mentions","urls","hashtags","text"]
data = data.drop(columns = to_drop)
data_eval = data_eval.drop(columns = to_drop)

## Save the data

In [ ]:
data.to_csv("data/training_v2.csv")
data_eval.to_csv("data/evaluation_v2.csv")

In [3]:
data_eval = pd.read_csv("data/evaluation_v2.csv")
data = pd.read_csv("data/training_v2.csv")

## First part : Classification phase

### Transform data

First, we need to transform the pd object into numpy array.

In [7]:
def to_list(L) :
    # input "[1,2,565454,1]"
    # ouput [1,2,565454,1]
    T = []
    for k in L :
        if not k in [" ",",","[","]"] :
            try :
                x = x*10 + int(k)
            except :
                x = int(k)
        else : 
            try :
                if k != ' ' :
                    T.append(x)
                    x = 0
            except : 
                count = 0
    return T

def from_pd_to_numpy(database,training=True) :
    X = np.zeros((database.shape[0],20))
    y = np.zeros(database.shape[0])
    for k in range(database.shape[0]) :
        current = database.iloc[k]
        if training : 
            y[k] = current.retweet_count
        else :
            y[k] = current.id
        X[k,0] = current.user_verified
        X[k,1] = current.user_statuses_count
        X[k,2] = current.user_followers_count
        X[k,3] = current.user_friends_count
        X[k,4] = current.num_hashtag
        X[k,5] = current.got_hashtag
        X[k,6] = current.num_at
        X[k,7] = current.got_at
        X[k,8] = current.num_link
        X[k,9] = current.got_link
        X[k,10] = current.length
        X[k,11] = current.contains_rt
        X[k,12] = current.weak
        X[k,13] = current.strong
        X[k,14] = current.is_upper
        X[k,15] = current.contains_excl
        X[k,16] = current.contains_per
        X[k,17] = current.contains_org
        X[k,18] = current.contains_gpe
        X[k,19] = current.zeros_pic
        """try : # if we read the database, it will be a string
            X[k,11:18] = to_list(current.day)
            X[k,18:42] = to_list(current.hour)
            X[k,42:54] = to_list(current.month)
        except :
            X[k,11:18] = current.day
            X[k,18:42] = current.hour
            X[k,42:54] = current.month"""
    
    return X,y   

X,y = from_pd_to_numpy(data)

### Classification

But now, we do not want yet to predict the number of retweets, but rather classify them. Therefore, we will transform *y* to define 3 classes. RT = 0, 0 < RT < 1000 and RT > 1000

In [160]:
def to_classify(y) :
    z = np.zeros(y.shape)
    for k in range(y.shape[0]) :
        f = y[k]
        if f > 0 :
            if f < 10 :
                z[k] = 1
            elif f < 100 :
                z[k] = 2
            elif f < 1000 :
                z[k] = 3
            elif f < 10000 :
                z[k] = 4
            else :
                z[k] = 5
    return z

classes = to_classify(y)

## Training dataset

X contains too much data around 0. We want to have as much tweet with class 0 than 5 in our training data.

In [266]:
def to_train(X,y,classes):
    max_class = np.amax(classes)
    count_class = {
        '0' : 0,
        '1' : 0,
        '2' : 0,
        '3' : 0,
        '4' : 0,
        '5' : 0,
        '6' :0
    }
    count = 0
    for k in range(X.shape[0]) :
        if classes[k] == max_class :
            count +=1
    X_train = np.zeros(X.shape)
    y_train = np.zeros(y.shape)
    classes_train = np.zeros(classes.shape)
    indice = 0
    print(count)
    for k in range(X.shape[0]) : 
        current_class = str(int(classes[k]))
        if count_class[current_class] < count*(max_class+ 1 -int(current_class))**0.4 :
            X_train[indice,:] = X[k,:]
            y_train[indice] = y[k]
            classes_train[indice] = classes[k]
            count_class[current_class]+=1
            indice+=1
        
    
    return X_train[:indice,:],y_train[:indice],classes_train[:indice]

X_train, y_train, classes_train = to_train(X,y,classes)

1810


In [267]:
X_train.shape

(17313, 20)

## Classification via random forest

In [268]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

clf = RandomForestClassifier(n_estimators=400,max_depth=6, random_state=1)
# clf = KNeighborsClassifier(n_neighbors=3)
# clf = svm.SVC()

clf.fit(X_train, classes_train)

RandomForestClassifier(max_depth=6, n_estimators=400, random_state=1)

## Classification via nn

In [ ]:
from sklearn.preprocessing import OneHotEncoder
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import train_test_split

ohe = OneHotEncoder()
y_cl = ohe.fit_transform(classes_train.reshape(-1,1)).toarray()

model_cl = Sequential()
model_cl.add(Dense(64, input_dim=20, activation="relu"))
model_cl.add(Dense(128, activation="relu"))
model_cl.add(Dense(32, activation="relu"))
model_cl.add(Dense(5, activation="softmax"))
model_cl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])
history = model_cl.fit(X_train, y_cl, epochs=1000, batch_size=64)

In [183]:
clf.predict(X[0:1,:]),clf.predict(X[0:1,:]).shape

(array([0.]), (1,))

In [269]:
output = clf.predict(X)

In [66]:
output_nn = model_cl.predict(X)

In [190]:
import random
for j in range(20) :
    k = random.randint(0,output.shape[0])
    print("Prediction : {} / Correct value : {} / Number of retweets : {}".format(int(output[k]),int(classes[k]),int(y[k])))

Prediction : 1 / Correct value : 1 / Number of retweets : 1
Prediction : 2 / Correct value : 2 / Number of retweets : 53
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 1 / Correct value : 1 / Number of retweets : 1
Prediction : 1 / Correct value : 1 / Number of retweets : 3
Prediction : 2 / Correct value : 2 / Number of retweets : 13
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 1 / Correct value : 1 / Number of retweets : 5
Prediction : 3 / Correct value : 2 / Number of retweets : 23
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 1 / Correct value : 1 / Number of retweets : 6
Prediction : 2 / Correct value : 2 / Number of retweets : 38
Prediction : 2 / Correct value : 1 / Number of retweets : 4
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 0 / Correct value : 0 / Number of retweets : 0
Prediction : 0 / Correct value : 0 /

In [270]:
def calculate_error(output,target) :
    error = 0
    for k in range(output.shape[0]) :
        if abs(int(output[k])- int(target[k])) != 0 :
            error += 1
    return error/output.shape[0]
        
        
def noRT(out) :
    if out == 0 :
        return 0
    return 3*10**(out-1)

In [271]:
calculate_error(output,classes)

0.20745534916345865

## Prediciton per class, but only ten choices

In [286]:
number_classes = int(np.amax(classes) + 1)
classes_entr = clf.predict(X)
# classes_entr = model_cl.predict(X)

In [287]:
def create_dataset_per_class_10(X,y,classes) :
    dataset = {}
    for k in range(X.shape[0]) :
        f = int(classes[k])
        # f = np.argmax(classes[k])
        try : 
            dataset[f] +=1
        except :
            dataset[f] = 1
    dataset2 = {}
    for k in range(int(np.amax(classes)+1)) :
    # for k in range(classes.shape[1]):
        dataset2[str(k)] = {}
        dataset2[str(k)]["X"] = np.zeros((dataset[k],X.shape[1]))
        dataset2[str(k)]["y"] = np.zeros(dataset[k])
        dataset2[str(k)]["value"] = np.zeros(dataset[k])
    
    indices = {
        '0' : 0,
        '1' : 0,
        '2' : 0,
        '3' : 0,
        '4' : 0,
        '5' : 0
    }
    for k in range(X.shape[0]) :
        f = str(int(classes[k]))
        # f = str(np.argmax(classes[k]))
        indice = indices[f]
        indices[f] +=1
        dataset2[f]["X"][indice,:] = X[k,:]
        dataset2[f]["y"][indice] = find_class(int(f),y[k])
        dataset2[f]["value"][indice] = y[k]
    return dataset2

def closest_int(x):
    low = int(x)
    high = low+1
    if x-low < high - x :
        return low
    return high

def find_class(classe,value) :
    if classe == 0 :
        return 0
    else :
        cl = closest_int(value/10**(classe-1))
        if cl < 10 :
            return cl
        return 9
    
dataset_per_class = create_dataset_per_class_10(X,y,classes_entr)

## Prediction per class

For each classes, we will build a predictor.

In [274]:
number_classes = int(np.amax(classes) + 1)
# classes_entr = clf.predict(X)
classes_entr = model_cl.predict(X)

In [275]:
def create_dataset_per_class(X,y,classes) :
    dataset = {}
    for k in range(X.shape[0]) :
        # f = int(classes[k])
        f = np.argmax(classes[k])
        try : 
            dataset[f] +=1
        except :
            dataset[f] = 1
    dataset2 = {}
    # for k in range(int(np.amax(classes)+1)) :
    for k in range(classes.shape[1]):
        dataset2[str(k)] = {}
        dataset2[str(k)]["X"] = np.zeros((dataset[k],X.shape[1]))
        dataset2[str(k)]["y"] = np.zeros(dataset[k])
    
    indices = {
        '0' : 0,
        '1' : 0,
        '2' : 0,
        '3' : 0,
        '4' : 0,
        '5' : 0
    }
    for k in range(X.shape[0]) :
        # f = str(int(classes[k]))
        f = str(np.argmax(classes[k]))
        indice = indices[f]
        indices[f] +=1
        dataset2[f]["X"][indice,:] = X[k,:]
        dataset2[f]["y"][indice] = y[k]
    return dataset2


dataset_per_class = create_dataset_per_class(X,y,classes_entr)

In [276]:
def get_dataset_class(dataset,class_to_get) :
    return dataset[str(class_to_get)]["X"], dataset[str(class_to_get)]["y"]

In [281]:
X_test, y_test = get_dataset_class(dataset_per_class,1)

In [282]:
X_test.shape,y_test.shape

((132653, 20), (132653,))

## All the models will be alike

*d* contains the training set and the model associated with the class.

In [288]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

def train_models(dataset_per_class) :
    ohe = OneHotEncoder()

    for k in dataset_per_class :
        if int(k) != 0 : # we do not train the first class as it is only a 0 prediction
            X_test, y_test = get_dataset_class(dataset_per_class,int(k))
            # neural network
            print(X_test.shape,y_test.shape)
            
            Train,a,train,b = train_test_split(X_test,y_test,test_size = 0.1)
            print(Train.shape,train.shape)
            train = ohe.fit_transform(train.reshape(-1,1)).toarray()
            if train.shape[1] < 10 :
                train = zaerniv = np.concatenate((train,np.zeros((train.shape[0],10-train.shape[1]), dtype=int)), axis=1)
            model = Sequential()
            model.add(Dense(64, input_dim=20, activation="sigmoid"))
            model.add(Dense(128, activation="relu"))
            model.add(Dense(256, activation="relu"))
            model.add(Dense(32, activation="relu"))
            model.add(Dense(10, activation="softmax"))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])
            history = model.fit(Train, train, epochs=1000, batch_size=64)
            # random forrest
            # model = RandomForestRegressor(n_estimators=200,max_depth=4, random_state=0)
            # model.fit(X_test,y_test)
            dataset_per_class[str(int(k))]["model"] = model
    return dataset_per_class

d = train_models(dataset_per_class)

(146119, 20) (146119,)
(131507, 20) (131507,)
Epoch 1/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7452 - categorical_crossentropy: 1.7452
Epoch 2/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7199 - categorical_crossentropy: 1.7199
Epoch 3/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7143 - categorical_crossentropy: 1.7143
Epoch 4/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7107 - categorical_crossentropy: 1.7107
Epoch 5/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7081 - categorical_crossentropy: 1.7081
Epoch 6/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7059 - categorical_crossentropy: 1.7059
Epoch 7/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7043 - categorical_crossentropy: 1.7043
Epoch 8/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.7025 - categorical_crossentropy: 1

2055/2055 [==============================] - 2s 1ms/step - loss: 1.6518 - categorical_crossentropy: 1.6518
Epoch 69/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6511 - categorical_crossentropy: 1.6511
Epoch 70/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6500 - categorical_crossentropy: 1.6500
Epoch 71/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6491 - categorical_crossentropy: 1.6491
Epoch 72/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6487 - categorical_crossentropy: 1.6487
Epoch 73/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6476 - categorical_crossentropy: 1.6476
Epoch 74/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6462 - categorical_crossentropy: 1.6462
Epoch 75/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6454 - categorical_crossentropy: 1.6454
Epoch 76/1000
2055/2055 [=====================

2055/2055 [==============================] - 3s 1ms/step - loss: 1.6038 - categorical_crossentropy: 1.6038
Epoch 136/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6033 - categorical_crossentropy: 1.6033
Epoch 137/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6029 - categorical_crossentropy: 1.6029
Epoch 138/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6029 - categorical_crossentropy: 1.6029
Epoch 139/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.6012 - categorical_crossentropy: 1.6012
Epoch 140/1000
2055/2055 [==============================] - 3s 2ms/step - loss: 1.6008 - categorical_crossentropy: 1.6008
Epoch 141/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6008 - categorical_crossentropy: 1.6008
Epoch 142/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.6007 - categorical_crossentropy: 1.6007
Epoch 143/1000
2055/2055 [=============

2055/2055 [==============================] - 2s 1ms/step - loss: 1.5773 - categorical_crossentropy: 1.5773
Epoch 203/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5770 - categorical_crossentropy: 1.5770
Epoch 204/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5774 - categorical_crossentropy: 1.5774
Epoch 205/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.5764 - categorical_crossentropy: 1.5764
Epoch 206/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.5751 - categorical_crossentropy: 1.5751
Epoch 207/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5758 - categorical_crossentropy: 1.5758
Epoch 208/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.5755 - categorical_crossentropy: 1.5755
Epoch 209/1000
2055/2055 [==============================] - 2s 1ms/step - loss: 1.5751 - categorical_crossentropy: 1.5751
Epoch 210/1000
2055/2055 [=============

2055/2055 [==============================] - 3s 1ms/step - loss: 1.5628 - categorical_crossentropy: 1.5628
Epoch 270/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5613 - categorical_crossentropy: 1.5613
Epoch 271/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5621 - categorical_crossentropy: 1.5621
Epoch 272/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5610 - categorical_crossentropy: 1.5610
Epoch 273/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5616 - categorical_crossentropy: 1.5616
Epoch 274/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5609 - categorical_crossentropy: 1.5609
Epoch 275/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5614 - categorical_crossentropy: 1.5614
Epoch 276/1000
2055/2055 [==============================] - 3s 1ms/step - loss: 1.5606 - categorical_crossentropy: 1.5606
Epoch 277/1000
2055/2055 [=============

...

28/28 [==============================] - 0s 2ms/step - loss: 0.0743 - categorical_crossentropy: 0.0743
Epoch 684/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0711 - categorical_crossentropy: 0.0711
Epoch 685/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0698 - categorical_crossentropy: 0.0698
Epoch 686/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0710 - categorical_crossentropy: 0.0710
Epoch 687/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0812 - categorical_crossentropy: 0.0812
Epoch 688/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0739 - categorical_crossentropy: 0.0739
Epoch 689/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0738 - categorical_crossentropy: 0.0738
Epoch 690/1000
28/28 [==============================] - 0s 1ms/step - loss: 0.0792 - categorical_crossentropy: 0.0792
Epoch 691/1000
28/28 [==============================] - 0s 1ms/step - l

28/28 [==============================] - 0s 2ms/step - loss: 0.0655 - categorical_crossentropy: 0.0655
Epoch 753/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0650 - categorical_crossentropy: 0.0650
Epoch 754/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0854 - categorical_crossentropy: 0.0854
Epoch 755/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0795 - categorical_crossentropy: 0.0795
Epoch 756/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0735 - categorical_crossentropy: 0.0735
Epoch 757/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0735 - categorical_crossentropy: 0.0735
Epoch 758/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0812 - categorical_crossentropy: 0.0812
Epoch 759/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0703 - categorical_crossentropy: 0.0703
Epoch 760/1000
28/28 [==============================] - 0s 2ms/step - l

28/28 [==============================] - 0s 2ms/step - loss: 0.0738 - categorical_crossentropy: 0.0738
Epoch 822/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0641 - categorical_crossentropy: 0.0641
Epoch 823/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0744 - categorical_crossentropy: 0.0744
Epoch 824/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0649 - categorical_crossentropy: 0.0649
Epoch 825/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0704 - categorical_crossentropy: 0.0704
Epoch 826/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0791 - categorical_crossentropy: 0.0791
Epoch 827/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0704 - categorical_crossentropy: 0.0704
Epoch 828/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0734 - categorical_crossentropy: 0.0734
Epoch 829/1000
28/28 [==============================] - 0s 2ms/step - l

28/28 [==============================] - 0s 2ms/step - loss: 0.0614 - categorical_crossentropy: 0.0614
Epoch 891/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0679 - categorical_crossentropy: 0.0679
Epoch 892/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0717 - categorical_crossentropy: 0.0717
Epoch 893/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0590 - categorical_crossentropy: 0.0590
Epoch 894/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0649 - categorical_crossentropy: 0.0649
Epoch 895/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0616 - categorical_crossentropy: 0.0616
Epoch 896/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0662 - categorical_crossentropy: 0.0662
Epoch 897/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0670 - categorical_crossentropy: 0.0670
Epoch 898/1000
28/28 [==============================] - 0s 2ms/step - l

28/28 [==============================] - 0s 2ms/step - loss: 0.0624 - categorical_crossentropy: 0.0624
Epoch 960/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0763 - categorical_crossentropy: 0.0763
Epoch 961/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0667 - categorical_crossentropy: 0.0667
Epoch 962/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0635 - categorical_crossentropy: 0.0635
Epoch 963/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0681 - categorical_crossentropy: 0.0681
Epoch 964/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0677 - categorical_crossentropy: 0.0677
Epoch 965/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0701 - categorical_crossentropy: 0.0701
Epoch 966/1000
28/28 [==============================] - 0s 2ms/step - loss: 0.0571 - categorical_crossentropy: 0.0571
Epoch 967/1000
28/28 [==============================] - 0s 2ms/step - l

## Now we need to compute these two steps on our evaluation data

### Transform eval_data

In [300]:
X_eval,id_eval = from_pd_to_numpy(data_eval,training=False)

### Classify the data

In [301]:
classes_eval = clf.predict(X_eval)
# classes_eval = model_cl.predict(X_eval)

In [302]:
classes_eval.shape

(285334,)

In [303]:
def create_dataset_eval_classes(X_eval,id_eval,eval_classes) :
    dataset = {}
    for k in range(X_eval.shape[0]) :
        try : 
            f = int(eval_classes[k]) # we have a prediction like 0 or 1 or 2
        except :
            f = np.argmax(eval_classes[k]) # we have something like [0.2,0.3,0.4,0.1]
        try : 
            dataset[f] +=1
        except :
            dataset[f] = 1
    dataset2 = {}
    for k in range(int(np.amax(eval_classes)+1)) :
    # for k in range(eval_classes.shape[1]) :
        dataset2[str(k)] = {}
        dataset2[str(k)]["X"] = np.zeros((dataset[k],X_eval.shape[1]))
        dataset2[str(k)]["id"] = np.zeros(dataset[k])
        dataset2[str(k)]["class"] = np.zeros(dataset[k])
    
    indices = {
        '0' : 0,
        '1' : 0,
        '2' : 0,
        '3' : 0,
        '4' : 0,
        '5' : 0
    }
    for k in range(X_eval.shape[0]) :
        try :
            f = str(int(eval_classes[k]))
        except :
            f = str(np.argmax(eval_classes[k])) # we have something like [0.2,0.3,0.4,0.1]
        indice = indices[f]
        indices[f] +=1
        dataset2[f]["X"][indice,:] = X_eval[k,:]
        dataset2[f]["id"][indice] = id_eval[k]
        dataset2[f]["class"][indice] = int(f)
    return dataset2

dataset_eval = create_dataset_eval_classes(X_eval,id_eval,classes_eval) 

### Use the previous model on each class

The prediction will be stored in *res*, which will then be ordered.

In [304]:
def evaluate_models(dataset_models,dataset_eval) :
    res = {}
    for indice in dataset_eval['0']["id"] :
        res[int(indice)] = 0
    for k in dataset_eval :
        if int(k) != 0 :
            X_pred = dataset_eval[k]["X"]
            prediction = dataset_models[k]["model"].predict(X_pred)
            for i in range(len(dataset_eval[k]["id"])) :
                res[int(dataset_eval[k]["id"][i])] = np.argmax(prediction[i])*10**(dataset_eval[k]["class"][i]-1)
    return res
    
res = evaluate_models(d,dataset_eval)

In [305]:
import collections

od = collections.OrderedDict(sorted(res.items()))

### Save the result

In [306]:
df = pd.DataFrame()
df["TweetID"] = od.keys()
df["NoRetweets"] = od.values()

In [296]:
df["count"] = data["retweet_count"]

In [297]:
df

,TweetID,NoRetweets,count
0,0,0.0,0
1,1,0.0,0
2,2,0.0,0
3,3,0.0,0
4,4,0.0,0
...,...,...,...
665772,665772,0.0,0
665773,665773,3.0,1
665774,665774,1.0,8
665775,665775,0.0,0


In [298]:
S = 0
for k in range(df.shape[0]) :
    print("Prediction : {} // Correct : {}".format())
    S += abs(int(df["NoRetweets"][k]) - int(df["count"][k]))
S = S / df.shape[0]

In [299]:
S

114.5104757298615

In [307]:
df.to_csv("results/res11.csv",index=False)

## One step

### Neural network